# Longitudinal trends of EHR concepts in pediatric patients

### Aims: predict the lwr which is 95% lower bound of the odds ratio quantified by the linear model.

### student informations

#### Surname: Emmanuel Anabel
#### Name: KECHAMEN
#### Department: Tip Muhendisligi
#### student number: 2210235507

In [92]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error

In [94]:
# loading the dataset 

df=pd.read_csv("concept_lm_trends.csv")
print(df)

         datatype  concept_id  \
0      Procedures     2007658   
1      Procedures     2102167   
2      Procedures     2000233   
3      Procedures     4289454   
4      Procedures     2105387   
...           ...         ...   
16455      Visits       32199   
16456      Visits    38004303   
16457      Visits        8870   
16458      Visits        8971   
16459      Visits        8584   

                                            concept_name      lwr     odds  \
0                      Other speech training and therapy  2845.90  4385.67   
1      Repair, intermediate, wounds of scalp, axillae...  3003.61  3431.79   
2            Injection of other agent, except neurolytic  2365.21  2381.31   
3              Venous catheterization for renal dialysis  1434.55  2043.66   
4      Closed treatment of proximal fibula or shaft f...  1739.94  1886.02   
...                                                  ...      ...      ...   
16455                          Information not available 

## Data Preparation steps

In [97]:
print(df.info()) 
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16460 entries, 0 to 16459
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   datatype      16460 non-null  object 
 1   concept_id    16460 non-null  int64  
 2   concept_name  16460 non-null  object 
 3   lwr           16460 non-null  float64
 4   odds          16460 non-null  float64
 5   upr           16460 non-null  float64
 6   rsq           16460 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 900.3+ KB
None
         concept_id           lwr          odds           upr           rsq
count  1.646000e+04  16460.000000  16460.000000  16460.000000  16460.000000
mean   1.159464e+07    -13.340246      2.922500     62.323829      0.422184
std    1.558744e+07    223.948574    210.019626    231.681895      0.334672
min    2.620000e+02  -9114.500000  -8475.410000  -6185.650000      0.000000
25%    1.314034e+06    -23.940000    -15.132500      4.1

In [99]:
print(df.isna().sum().sum()) # verifying if there are missing values (NaN) qnd how many they are

0


In [101]:
# converting datas(string) in Datatype columns into numbers(integers)

df["num_col(datatype)"]=pd.Categorical(df["datatype"]).codes
df["num_col(concept_name)"]=pd.Categorical(df["concept_name"]).codes
print(df.head())

     datatype  concept_id                                       concept_name  \
0  Procedures     2007658                  Other speech training and therapy   
1  Procedures     2102167  Repair, intermediate, wounds of scalp, axillae...   
2  Procedures     2000233        Injection of other agent, except neurolytic   
3  Procedures     4289454          Venous catheterization for renal dialysis   
4  Procedures     2105387  Closed treatment of proximal fibula or shaft f...   

       lwr     odds       upr   rsq  num_col(datatype)  num_col(concept_name)  
0  2845.90  4385.67  10713.82  0.32                  3                   8774  
1  3003.61  3431.79   5392.21  0.75                  3                  10102  
2  2365.21  2381.31   2447.98  1.00                  3                   6639  
3  1434.55  2043.66   4560.09  0.40                  3                  11798  
4  1739.94  1886.02   2546.74  0.80                  3                   3222  


In [103]:
print(df[df.duplicated()])

Empty DataFrame
Columns: [datatype, concept_id, concept_name, lwr, odds, upr, rsq, num_col(datatype), num_col(concept_name)]
Index: []


In [105]:
df=df.drop(['datatype','concept_name'], axis=1)
print(df)
print(df.info())

       concept_id      lwr     odds       upr   rsq  num_col(datatype)  \
0         2007658  2845.90  4385.67  10713.82  0.32                  3   
1         2102167  3003.61  3431.79   5392.21  0.75                  3   
2         2000233  2365.21  2381.31   2447.98  1.00                  3   
3         4289454  1434.55  2043.66   4560.09  0.40                  3   
4         2105387  1739.94  1886.02   2546.74  0.80                  3   
...           ...      ...      ...       ...   ...                ...   
16455       32199   -71.47   -63.40    -45.27  0.75                  4   
16456    38004303  -123.38   -71.16     47.76  0.03                  4   
16457        8870   -89.31   -73.48    -37.93  0.38                  4   
16458        8971   -99.19   -88.12    -63.67  0.76                  4   
16459        8584  -112.45   -92.89    -49.41  0.53                  4   

       num_col(concept_name)  
0                       8774  
1                      10102  
2                 

In [107]:
# identifying data points significantly different from the rest by computing the Z-scores

from scipy import stats

z=np.abs(stats.zscore(df))
print(df.head())

   concept_id      lwr     odds       upr   rsq  num_col(datatype)  \
0     2007658  2845.90  4385.67  10713.82  0.32                  3   
1     2102167  3003.61  3431.79   5392.21  0.75                  3   
2     2000233  2365.21  2381.31   2447.98  1.00                  3   
3     4289454  1434.55  2043.66   4560.09  0.40                  3   
4     2105387  1739.94  1886.02   2546.74  0.80                  3   

   num_col(concept_name)  
0                   8774  
1                  10102  
2                   6639  
3                  11798  
4                   3222  


In [109]:
selected = df.iloc[np.where(z > 3)[0]].index  
print(selected) 

Index([    0,     0,     0,     1,     1,     1,     2,     2,     2,     3,
       ...
       10932, 10933, 10933, 10933, 10934, 10934, 10934, 10935, 16432, 16432],
      dtype='int64', length=702)


In [111]:
dfN=df.drop(selected, axis=0, inplace=True)
print(dfN)  

None


In [113]:
# it turns out that all the computed z values for this dataset in particular are greater than 3 so i won't be able to remove the line that contains values significantly different from the rest so i'ill will leave the dataframe as it is and jump to the next step

## Normalisation

In [116]:
df=(df-df.min())/(df.max()-df.min())
print(df)

       concept_id       lwr      odds       upr   rsq  num_col(datatype)  \
29       0.087813  0.976803  0.990463  0.934286  0.90               0.75   
30       0.091087  1.000000  0.989333  0.845112  1.00               0.75   
31       0.043348  0.960451  0.978361  0.943375  0.92               0.75   
35       0.045598  0.937426  0.965061  0.987148  0.72               0.75   
36       0.059385  0.932235  0.927515  0.821513  0.99               0.75   
...           ...       ...       ...       ...   ...                ...   
16455    0.000690  0.500802  0.477218  0.411736  0.75               1.00   
16456    0.821044  0.458298  0.470628  0.479981  0.03               1.00   
16457    0.000186  0.486195  0.468658  0.417120  0.38               1.00   
16458    0.000188  0.478105  0.456225  0.398238  0.76               1.00   
16459    0.000180  0.467247  0.452174  0.408699  0.53               1.00   

       num_col(concept_name)  
29                  0.116201  
30                  0.395

In [118]:
# separating the dataset in features(x) ansd target(y)

x=df.drop(['rsq'], axis=1)
y=df['rsq'].values

In [120]:
# splitting the data into training and testing set

x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2, random_state=42)

In [122]:
# training the linear regression model

lr=LinearRegression()
lr.fit(x_train,y_train)

LinearRegression()

In [124]:
# making prediction of the set 

y_pred=lr.predict(x_test)

In [126]:
print(lr.intercept_,lr.coef_)

1.3335696585250263 [ -0.03790107 -20.2468943   25.95327374  -7.55453765   0.31939413
  -0.10061107]


In [128]:
# computation of metrics 

mse=mean_squared_error(y_test,y_pred)
rmse=np.sqrt(mse)
mape=mean_absolute_percentage_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)

In [130]:
# printing the results of the metrics
print(f'mean squared error: {mse:2f}')
print(f'mean squared root error: {rmse:2f}')
print(f'mean absolute percentage error: {mape:2f}')
print(f'mean absolute error: {mae:2f}')
print(f'R-squared: {r2:2f}')

mean squared error: 0.096381
mean squared root error: 0.310452
mean absolute percentage error: 91534522658460.343750
mean absolute error: 0.270811
R-squared: 0.129857


## Polynomial Regression

In [133]:
# creating polynomial caracteristics
pol=PolynomialFeatures(degree=3)
x_pol_train=pol.fit_transform(x_train)
x_pol_test=pol.fit_transform(x_test)

In [135]:
# creating the regression model
plr=LinearRegression()
plr.fit(x_pol_train,y_train)

LinearRegression()

In [137]:
# making prediction
y_pol_pred=plr.predict(x_pol_test)

In [139]:
# computation of metrics
pol_mse=mean_squared_error(y_test,y_pol_pred)
pol_rmse=np.sqrt(mse)
pol_mape=mean_absolute_percentage_error(y_test,y_pol_pred)
pol_mae=mean_absolute_error(y_test,y_pol_pred)
pol_r2=r2_score(y_test,y_pol_pred)

In [141]:
# printing the results of the metrics
print(f'mean squared error: {pol_mse:2f}')
print(f'mean squared root error: {pol_rmse:2f}')
print(f'mean absolute percentage error: {pol_mape:2f}')
print(f'mean absolute error: {pol_mae:2f}')
print(f'R-squared: {pol_r2:2f}')

mean squared error: 0.063997
mean squared root error: 0.310452
mean absolute percentage error: 66318962430981.554688
mean absolute error: 0.204351
R-squared: 0.422221


# Decision Tree

In [144]:
from sklearn.tree import DecisionTreeRegressor

In [146]:

dt=DecisionTreeRegressor()
dt.fit(x_train,y_train) # train the decision tree model with known data
dt_pred=dt.predict(x_test)  # presict the the outcomes

In [148]:
# computation of metrics
dt_mse=mean_squared_error(y_test,dt_pred)
dt_rmse=np.sqrt(mse)
dt_mape=mean_absolute_percentage_error(y_test,dt_pred)
dt_mae=mean_absolute_error(y_test,dt_pred)
dt_r2=r2_score(y_test,dt_pred)

In [150]:
# printing the results of the metrics
print(f'mean squared error: {dt_mse:2f}')
print(f'mean squared root error: {dt_rmse:2f}')
print(f'mean absolute percentage error: {dt_mape:2f}')
print(f'mean absolute error: {dt_mae:2f}')
print(f'R-squared: {dt_r2:2f}')

mean squared error: 0.027311
mean squared root error: 0.310452
mean absolute percentage error: 1047673610586.083130
mean absolute error: 0.104712
R-squared: 0.753430


#  Random Forest

In [153]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf=RandomForestRegressor()
rf.fit(x_train,y_train) # train the decision tree model with known data
rf_pred=rf.predict(x_test)  # presict the the outcomes

In [ ]:
# computation of metrics
rf_mse=mean_squared_error(y_test,rf_pred)
rf_rmse=np.sqrt(mse)
rf_mape=mean_absolute_percentage_error(y_test,rf_pred)
rf_mae=mean_absolute_error(y_test,rf_pred)
rf_r2=r2_score(y_test,rf_pred)

In [ ]:
# printing the results of the metrics
print(f'mean squared error: {rf_mse:2f}')
print(f'mean squared root error: {rf_rmse:2f}')
print(f'mean absolute percentage error: {rf_mape:2f}')
print(f'mean absolute error: {rf_mae:2f}')
print(f'R-squared: {rf_r2:2f}')